Currently the requirements to get IV estimates are:
 - Your player level (which must remain constant as you're leveling this pokémon)
 - The pokémon's CP, HP, and stardust requirements for the next level
 - Every additional line of data added must be from only one power up (i.e. don't power up many times without also including those values in the table
 
 In the near future I will add the ability to:
 - Specify the pokémon's level or a range of levels to add in as a prior
 - Specify whether the pokémon was caught in the "wild" or from an "egg", and whether you've powered it up or not yet 
 - Put in any data and indicate if any power-up levels have been skipped


### What is your player level?

In [17]:
player_level = 15

### What is the pokémon's name? (type in quote marks)

In [18]:
pokemonName= "sandshrew"

### First data set! Put in the pokémon's CP, HP, and stardust to level

In [34]:
#data=(232,42,1900) #pidgey
#data=(1095,131,1300) #evil lapras
data=(219,47,1000) #mudball Jnr lv 1
#data=(230,48,1000) #mudball Jnr lv 2
#data=(241,50,1300) #mudball Jnr lv 3
#data=(287,36,1900) #ekans1
#data=(246,35,1300) #ekans2
#data=(174,53,600) #rhyhorn lv 1
#data=(191,55,600) #rhyhorn lv 2
#data=(209,58,600) #rhyhorn lv 3
#data=(226,60,800) #rhyhorn lv 4
#data=(727,83,1300) #Hypno lv 1
#data=(758,85,1300) #Hypno lv 2
#data=(788,87,1600) #Hypno lv 3
#data=(540,60,1300) #Weepingbell lv 1
#data=(563,61,1300) #Weepingbell lv 2
#data=(585,62,1600) #Weepingbell lv 3
#data=(237,45,1600) #pidgey lv 1
#data=(246,46,1600) #pidgey lv 2
#data=(255,47,1600) #pidgey lv 3
#data=(264,48,1600) #pidgey lv 4
#data=(272,49,1900) #pidgey lv 5
#data=(283,49,1900) #pidgey lv 6
#data=(292,50,1900) #pidgey lv 7
#data=(301,51,1900) #pidgey lv 8 (but can't level)

##### Code is going to chug along now

In [84]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pokedicts as pd


#HP IV calc is easiest
#HP = (basestamina + ivstamina)*TotalCPMultiplier

#CP = MAX(10,FLOOR(HP**0.5*Attack*Def**0.5/10))
#where
#Attack = (baseatt + ivatt)*TotalCPMultiplier
#Defense= (basedef + ivdef)*TotalCPMultiplier
#TCpM =
#CP= MAX(10,FLOOR( (basesta+ivsta)**2 * (basedef+ivatt) * (basedef+ivdef)**2 ) * TCpM**2/10)
#CP=f(ivsta,ivatt,ivdef)TCpM(level)
#HP=h(ivsta)

#I can calculate p(CP|ivsta,ivatt,ivdef,level), p(HP|ivsta,level), p(stardust|level)

#p_stardust is already set and is again dependent on player level
#p(stardust|player)=p(player|stardust)*p(stardust)/p(player)
#p(player) is a complete given so p(player)=1, equation reduces to
# # p(stardust|player) as this is calculable yay!


#full list of stardusts:
#allPossibleStardusts = np.array([levelToStardust[x] for x in map_level])

#BUT some stardusts are impossible given the player level, mark some as impossible
#allPossibleStardusts[(map_level > playerLevel*2)-1] = -1
#setStardusts = np.sort(np.array(list(set(allPossibleStardusts).remove(-1))))
#
#print allPossibleStardusts
#print setStardusts
#p_stardust = np.array([allPossibleStardusts.count(x) for x in setStardusts],dtype=float)
#p_stardust/=np.float(np.sum(p_stardust))
#
#print p_stardust


#Easiest thing to do would be to calculate all values? 16 x 16 x 16 x 79 pokemon levels?
#Can do 16x16x16 itself and then use matrix multiplication to work out the other




############## MODEL CONSTRUCTION (all possible results) #####################

# 1d map all possible pokemon levels
map1d_level=(np.arange(79)+2.)/2.

# 1d map all possible IVs
map1d_ivatt=np.arange(16)
map1d_ivdef=np.arange(16)
map1d_ivsta=np.arange(16)


#all CP modifier values, 1d and 4d
map1d_TCpM= np.array([pd.levelToTCpM[i] for i in map1d_level])

#all possible attack, def, stamina values (all possible levels)
map1d_att=map1d_ivatt+baseAtt
map1d_def=map1d_ivdef+baseDef
map1d_sta=map1d_ivsta+baseSta

#Making 4D array of variables and partial calculations
map_sta, map_att, map_def, map_TCpM=np.meshgrid(map1d_sta,map1d_att,\
                                                 map1d_def,map1d_TCpM, indexing='ij')

map_ivsta, map_ivatt, map_ivdef, map_level=np.meshgrid(map1d_ivsta,map1d_ivatt,\
                                                 map1d_ivdef,map1d_level, indexing='ij')



#all possible CP values 
allPossibleCP = np.floor((map_sta**0.5*map_att*map_def**0.5*map_TCpM**2)/10.)
allPossibleCP[allPossibleCP<10]=10

#all possible HP values
allPossibleHP =np.floor(map_sta*map_TCpM) 
allPossibleHP[allPossibleHP<10]=10

#all possible Stardust values
allPossibleSD=np.array([pd.levelToStardust[i] \
                        for i in map_level.ravel()]).reshape(map_level.shape)


#Print out some fun facts from the model 
#print "min/max possible stardust", np.min(allPossibleSD), np.max(allPossibleSD)
print "max possible CP for any",pokemonName,np.max(allPossibleCP)
print "max possible HP for any",pokemonName,np.max(allPossibleHP)


#From those three 4D arrays we have a model in StAtDeLv space tah dah




############## INITIAL PRIOR CONSTRUCTION ##################
#Construct initial (uninformative) prior:
prior_STATDELV=np.ones(map_sta.shape,dtype="float")

#Immediately apply player level prior (could just do the stardust but want to decouple)
prior_STATDELV[ map_level > player_level] = 0

#Stardust counts as an observation so won't apply as an initial prior

#normalise prior
prior_STATDELV/=np.sum(prior_STATDELV)




############# LOOP OVER THE DATA ####################


LH_CP=(allPossibleCP == dataCP) #this is in STATDELV space
LH_HP=(allPossibleHP == dataHP)
LH_SD=(allPossibleSD == dataSD)


LH_Joint=np.array(LH_CP*LH_HP*LH_SD,dtype="float" )

print "\nNumber of possible stamina, attack and defense ivs, and levels: ", np.sum(LH_Joint)
#Fold in the previous posterior as a prior

#print "possible iv HP values given HP:",list(set(map_sta[LH_HP]-baseSta))
#print "possible iv HP values given SD:",list(set(map_sta[LH_SD]-baseSta))
#print "possible levels given HP: ",list(set(map_level[LH_HP]))
#print "possible levels given SD: ",list(set(map_level[LH_SD]))

#Fold in the previous posterior (or initial prior) as a prior 
LH_Joint*=prior_STATDELV
LH_Joint/=np.sum(LH_Joint)

#Need to pass this posterior to prior_StAtDeLv
prior_STATDELV=LH_Joint

############## Finding Different Stats :) ################

LH_ind=np.where(LH_Joint > 0)
#find all cominations of ivs and lvs
possible_STATDELV=np.array([map_ivsta[LH_ind],\
                            map_ivatt[LH_ind],\
                            map_ivdef[LH_ind],\
                            map_level[LH_ind]]).T



print "\nAll possible combinations:"
for i, vals in enumerate(possible_STATDELV):
    print "level:", vals[3], \
        "  ivs: stamina: ", vals[0], \
        "  attack:", vals[1], \
        "  defense:",vals[2], \
        "  probability:",np.round(LH_Joint[LH_ind[0][i],LH_ind[1][i],LH_ind[2][i],LH_ind[3][i]]*100,3),'%'
        
#Perfect IVs
print "\n\nProbability of perfect ivs (given any possible level): ",\
                    np.round(np.sum(LH_Joint[15,15,15,:])*100 , 3),'%'
    
#Two perfect IVs 
test_ind=np.where(((map_ivsta==15) & (map_ivatt==15)) |\
                  ((map_ivsta==15) & (map_ivdef==15)) |\
                  ((map_ivatt==15) & (map_ivdef==15)))
print "Probability of two perfect ivs (given any possible level): ",\
                    np.round(100*np.sum(LH_Joint[test_ind]), 3),'%'

#One perfect IV
test_ind=np.where(  (map_ivsta==15) | (map_ivatt==15) | (map_ivdef==15)  )
print "Probability of one perfect iv (given any possible level): ",\
                    np.round(100*np.sum(LH_Joint[test_ind]), 3),'%'

#Three top two IVs
test_ind=np.where(  (map_ivsta>=14) & (map_ivatt>=14) & (map_ivdef>=14)  )
print "Probability of three 14/15 ivs (given any possible level): ",\
                    np.round(100*np.sum(LH_Joint[test_ind]), 3),'%'

#Two top two IVs
test_ind=np.where(((map_ivsta>=14) & (map_ivatt>=14)) |\
                  ((map_ivsta>=14) & (map_ivdef>=14)) |\
                  ((map_ivatt>=14) & (map_ivdef>=14)))
print "Probability of two 14/15 ivs (given any possible level): ",\
                    np.round(100*np.sum(LH_Joint[test_ind]), 3),'%'





#Giving stats for each level     
print "\nNow considering on a level-by-level basis: "
for lv in list(set(map_level[LH_ind])):
    print "For level "+str(lv)+":"
    

max possible CP for any sandshrew 798.0
max possible HP for any sandshrew 90.0

Number of possible stamina, attack and defense ivs, and levels:  15.0

All possible combinations:
level: 15.0   ivs: stamina:  0.0   attack: 9.0   defense: 15.0   probability: 6.667 %
level: 15.0   ivs: stamina:  0.0   attack: 10.0   defense: 13.0   probability: 6.667 %
level: 15.0   ivs: stamina:  0.0   attack: 11.0   defense: 10.0   probability: 6.667 %
level: 15.0   ivs: stamina:  0.0   attack: 12.0   defense: 8.0   probability: 6.667 %
level: 15.0   ivs: stamina:  0.0   attack: 14.0   defense: 3.0   probability: 6.667 %
level: 15.0   ivs: stamina:  0.0   attack: 15.0   defense: 1.0   probability: 6.667 %
level: 14.5   ivs: stamina:  1.0   attack: 12.0   defense: 15.0   probability: 6.667 %
level: 14.5   ivs: stamina:  1.0   attack: 13.0   defense: 13.0   probability: 6.667 %
level: 14.5   ivs: stamina:  1.0   attack: 14.0   defense: 10.0   probability: 6.667 %
level: 14.5   ivs: stamina:  1.0   attack: 

In [7]:
## Working out probabilities of finding these data values randomly


#Setting up the real CP/HP/SD values, their indices and a dictionary
map1d_CP=np.arange(800,dtype='float')+10
CPtoIndex=dict(zip(map1d_CP,np.arange(len(map1d_CP),dtype='float')))
               
map1d_HP=np.arange(200,dtype='float')+10
HPtoIndex=dict(zip(map1d_HP,np.arange(len(map1d_HP),dtype='float')))

map1d_SD=np.sort(np.array(list(set(np.array(\
                    [levelToStardust[lvl] for lvl in map1d_level],dtype='float')))))
SDtoIndex=dict(zip(map1d_SD, np.arange(len(map1d_SD),dtype='float')))


countsCPHPSD=np.zeros((map1d_CP.shape[0],map1d_HP.shape[0],map1d_SD.shape[0]),dtype="float")

flatCP=allPossibleCP.flatten()
flatHP=allPossibleHP.flatten()
flatSD=allPossibleSD.flatten()


flatCPHPSD=np.array([flatCP,flatHP,flatSD]).T


print "\n\ncalculating bayesian evidence??? (few seconds)"
for CPHPSD in flatCPHPSD:
    #print CPtoIndex[CPHPSD[0]], HPtoIndex[CPHPSD[1]],SDtoIndex[CPHPSD[2]]
    countsCPHPSD[ CPtoIndex[CPHPSD[0]] , HPtoIndex[CPHPSD[1]] , SDtoIndex[CPHPSD[2]] ] +=1

#marginalise
countsCPHPSD/=np.sum(countsCPHPSD)

#Collapse counts down along axes)
countsCP=[np.sum(countsCPHPSD[i,:,:]) for i in range(len(countsCPHPSD[:,0,0]))]
countsHP=[np.sum(countsCPHPSD[:,i,:]) for i in range(len(countsCPHPSD[0,:,0]))]
countsSD=[np.sum(countsCPHPSD[:,:,i]) for i in range(len(countsCPHPSD[0,0,:]))]


plt.plot(map1d_CP,countsCP)
plt.show()

plt.plot(map1d_HP,countsHP)
plt.show()

plt.plot(map1d_SD,countsSD)
plt.show()

NameError: name 'levelToStardust' is not defined